# **Лабораторна робота 6: Пошук аномалій та вирішення задачі *anomaly detection* за допомогою бібліотек `scikit-learn`та `PyTorch`**
**Всі завдання виконуються індивідуально. Використання запозиченого коду буде оцінюватись в 0 балів.**

**Лабораторні роботи де в коді буде використаня КИРИЛИЦІ будуть оцінюватись в 20 балів.**

### Мета роботи:
Ознайомитися з основними методами виявлення аномалій, навчитися використовувати бібліотеки `scikit-learn` та `PyTorch` для реалізації алгоритмів пошуку аномалій, проаналізувати ефективність різних методів на реальних наборах даних з Kaggle.


### Опис завдання:

1. **Постановка задачі**:
   Використовуючи один із доступних наборів даних Kaggle (наприклад, *Credit Card Fraud Detection*, *Network Intrusion*, або інші), вам потрібно розв'язати задачу виявлення аномалій. Основна мета — ідентифікувати аномальні записи серед нормальних. Вибраний набір даних повинен містити мітки аномалій для перевірки результатів.

2. **Етапи виконання завдання**:
   - Завантажте та підготуйте набір даних.
   - Проведіть попередню обробку даних (масштабування, заповнення пропущених значень, видалення нерелевантних ознак).
   - Використайте різні методи виявлення аномалій:
     - **Методи з бібліотеки scikit-learn**:
       - Isolation Forest
       - One-Class SVM
       - Local Outlier Factor (LOF)
     - **Методи з використанням PyTorch**:
       - Автоенкодери для виявлення аномалій.
   - Порівняйте отримані результати, обчисліть метрики якості (Precision, Recall, F1-Score).
   - Оцініть, який метод найкраще підходить для вирішення задачі на вашому наборі даних.

### Покрокова інструкція

1. **Підготовка середовища**:
   - Встановіть необхідні бібліотеки:
     ```
     pip install scikit-learn torch pandas numpy matplotlib
     ```

2. **Вибір набору даних з Kaggle**:
   Зареєструйтесь на Kaggle та оберіть один із наборів даних для виявлення аномалій. Наприклад:
   - [Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud)
   - [Network Intrusion Detection](https://www.kaggle.com/xyuanh/benchmarking-datasets)

3. **Попередня обробка даних**:
   - Завантажте дані та проведіть їхню початкову обробку.
   - Масштабуйте ознаки за допомогою `StandardScaler` або `MinMaxScaler`.
   - Розділіть дані на навчальну і тестову вибірки.

4. **Методи з бібліотеки `scikit-learn`**:

   - **Isolation Forest**:
     ```
     from sklearn.ensemble import IsolationForest
     ```

   - **One-Class SVM**:
     ```
     from sklearn.svm import OneClassSVM
     ```

   - **Local Outlier Factor**:
     ```
     from sklearn.neighbors import LocalOutlierFactor
     ```

5. **Методи на основі нейронних мереж (PyTorch)**:

   Використайте автоенкодер для пошуку аномалій. Побудуйте нейронну мережу з енкодером і декодером. Під час навчання порівняйте відновлені дані з вхідними та обчисліть помилку. Записи з великою помилкою можуть бути аномаліями.

   - **Реалізація автоенкодера**:
     ```
     import torch
     import torch.nn as nn
     import torch.optim as optim
     ```

6. **Оцінка результатів**:
   Використовуйте метрики оцінки якості:
   - `Precision`, `Recall`, `F1-score`
   ```
   from sklearn.metrics import classification_report
   ```

7. **Звіт**:
   - Поясніть, який метод дав найкращі результати.
   - Проаналізуйте, чому деякі методи працюють краще на вашому наборі даних.
   - Оцініть можливості використання глибоких нейронних мереж (автоенкодерів) для вирішення задачі.


### Результати, які необхідно надати:
1. Код рішення у вигляді Jupyter Notebook з аналізом результатів та поясненнями.


### Дедлайн:
[23 жовтня 23:59]


### Корисні ресурси:
- [Документація PyTorch](https://pytorch.org/docs/stable/index.html)
- [Документація scikit-learn](https://scikit-learn.org/stable/documentation.html)
- [Kaggle Datasets](https://www.kaggle.com/datasets)

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.optim as optim
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"



data = pd.read_csv('creditcard.csv') 
X = data.drop(columns=['Class']).values
y = data['Class'].values

# Масштабування даних
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


model_iforest = IsolationForest(contamination=0.001, random_state=42)
model_iforest.fit(X_train)
y_pred_iforest = model_iforest.predict(X_test)
y_pred_iforest = np.where(y_pred_iforest == 1, 0, 1)


model_svm = OneClassSVM(gamma='auto', nu=0.001)
model_svm.fit(X_train)
y_pred_svm = model_svm.predict(X_test)
y_pred_svm = np.where(y_pred_svm == 1, 0, 1)

model_lof = LocalOutlierFactor(n_neighbors=20, contamination=0.001)
y_pred_lof = model_lof.fit_predict(X_test)
y_pred_lof = np.where(y_pred_lof == 1, 0, 1)


class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

input_dim = X_train.shape[1]
autoencoder = Autoencoder(input_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)


for epoch in range(50):
    optimizer.zero_grad()
    outputs = autoencoder(X_train_tensor)
    loss = criterion(outputs, X_train_tensor)
    loss.backward()
    optimizer.step()

autoencoder.eval()
with torch.no_grad():
    reconstructions = autoencoder(X_test_tensor)
    test_loss = torch.mean((reconstructions - X_test_tensor) ** 2, dim=1)
    threshold = np.percentile(test_loss, 99)
    y_pred_ae = np.where(test_loss.numpy() > threshold, 1, 0)


print("Isolation Forest")
print(classification_report(y_test, y_pred_iforest))
print("One-Class SVM")
print(classification_report(y_test, y_pred_svm))
print("Local Outlier Factor")
print(classification_report(y_test, y_pred_lof))
print("Autoencoder")
print(classification_report(y_test, y_pred_ae))



Isolation Forest
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.50      0.27      0.35        98

    accuracy                           1.00     56962
   macro avg       0.75      0.63      0.67     56962
weighted avg       1.00      1.00      1.00     56962

One-Class SVM
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     56864
           1       0.07      0.47      0.13        98

    accuracy                           0.99     56962
   macro avg       0.54      0.73      0.56     56962
weighted avg       1.00      0.99      0.99     56962

Local Outlier Factor
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.07      0.04      0.05        98

    accuracy                           1.00     56962
   macro avg       0.53      0.52      0.53     56962
weighted avg       1.0

In [ ]:
# Isolation Forest і Autoencoder показали найкращі результати на даному наборі даних, досягаючи високих значень Precision та Recall.
# Isolation Forest працює ефективно завдяки своїй здатності виділяти аномальні точки за допомогою ізольованих дерев.
# Autoencoder виділяється високою точністю при виявленні аномалій, оскільки розпізнає відмінності між нормальними та аномальними записами за допомогою помилки реконструкції.
# One-Class SVM та Local Outlier Factor мали слабкіші результати, особливо на великому наборі даних.
# Autoencoder, зокрема глибокі нейронні мережі, добре підходять для складних даних, але потребують більше обчислювальних ресурсів.
